In [7]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds
sns.set_style

<function seaborn.rcmod.set_style(style=None, rc=None)>

In [8]:
mnist_dataset, mnist_info =tfds.load(name='mnist',with_info=True,as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\mnist\3.0.1.incomplete3D529M\mnist-train.tfrecord*...:   0%|          | 0/6000…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\mnist\3.0.1.incomplete3D529M\mnist-test.tfrecord*...:   0%|          | 0/10000…

Dataset mnist downloaded and prepared to ~\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [9]:
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']

num_validation_samples=0.1 * mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)

In [11]:
def scale (image, label):
    image=tf.cast(image,tf.float32)
    image=image/255.
    return image,label
scaled_train_and_validation_data=mnist_train.map(scale)
test_data=mnist_test.map(scale)

In [12]:
BUFFER_SIZE=10000
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE=100
train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)

validation_inputs, validation_targets= next(iter(validation_data))

In [41]:
input_size=784
output_size=10
hidden_layers_size=200

model= tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layers_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layers_size,activation='tanh'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
                        ])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [42]:
NUMBER_EPOCHS=5
model.fit(train_data,epochs=NUMBER_EPOCHS,validation_data=(validation_inputs, validation_targets),verbose=2)

Epoch 1/5
540/540 - 3s - loss: 0.2552 - accuracy: 0.9235 - val_loss: 0.1287 - val_accuracy: 0.9618 - 3s/epoch - 6ms/step
Epoch 2/5
540/540 - 2s - loss: 0.0968 - accuracy: 0.9704 - val_loss: 0.0849 - val_accuracy: 0.9745 - 2s/epoch - 3ms/step
Epoch 3/5
540/540 - 2s - loss: 0.0652 - accuracy: 0.9802 - val_loss: 0.0581 - val_accuracy: 0.9835 - 2s/epoch - 3ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0469 - accuracy: 0.9852 - val_loss: 0.0605 - val_accuracy: 0.9818 - 2s/epoch - 3ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0357 - accuracy: 0.9887 - val_loss: 0.0482 - val_accuracy: 0.9872 - 2s/epoch - 3ms/step


In [45]:
test_loss, test_accuracy= model.evaluate(test_data)

1/1 [==============================] - 0s 165ms/step - loss: 0.0822 - accuracy: 0.9761


In [60]:
print('Test loss = {0:.2f} , Test accuracy = {1:.2f} %'.format(test_loss,(test_accuracy*100.)))

Test loss = 0.08 , Test accuracy = 97.61 %
